In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from collections import OrderedDict
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm
nlp = en_core_web_sm.load()
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preprocessing

In [5]:
#DATA PRE-PROCESSING
#loading the dataset to a pandas DataFrame
def Data_PreProcessing():
    news_dataset = pd.read_csv('/content/drive/MyDrive/fakeNews dataSet/train.csv')
    # replacing the null values with empty string
    news_dataset=news_dataset.fillna('') 
    #merging the text and the news title
    news_dataset['content']=news_dataset['title']+' '+news_dataset['author']
    return news_dataset

In [6]:
news_dataset=Data_PreProcessing()

In [7]:
#Steaming:
#steaming is the process of reducing a word to its Root Word
#exemple actor,actress,acting -->act

port_stem=PorterStemmer()
def preProcessing(text):
    text_preprocessed=re.sub('[^a-zA-Z]',' ', text)
    text_preprocessed=text_preprocessed.lower()
    text_preprocessed=text_preprocessed.split()
    text_preprocessed=[port_stem.stem(word) for word in text_preprocessed if not word in stopwords.words('english')]
    text_preprocessed=' '.join(text_preprocessed)
    return text_preprocessed


# Making a file containing the concatenation of all the texts

In [8]:
def MergeInFile(fileName,DataSize):
    f = open(fileName, "a")
    for i in range(DataSize):
        print(i)
        text= preProcessing(news_dataset['content'][i])
        f.write(text)
    print('fin')

In [9]:
MergeInFile("Docxxx.txt",1000)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
def ReadDataFromFile(fileName):
    f = open(fileName, "r")
    text=f.read()
    return text

In [11]:
text=ReadDataFromFile("/content/Docxxx.txt")
print(text)

hous dem aid even see comey letter jason chaffetz tweet darrel lucuflynn hillari clinton big woman campu breitbart daniel j flynntruth might get fire consortiumnew comcivilian kill singl us airstrik identifi jessica purkissiranian woman jail fiction unpublish stori woman stone death adulteri howard portnoyjacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart daniel nussbaumlife life luxuri elton john favorit shark pictur stare long transcontinent flightbeno hamon win french socialist parti presidenti nomin new york time alissa j rubinexcerpt draft script donald trump q ampa black church pastor new york timeback channel plan ukrain russia courtesi trump associ new york time megan twohey scott shaneobama organ action partner soro link indivis disrupt trump agenda aaron kleinbbc comedi sketch real housew isi caus outrag chri tomlinsonrussian research discov secret nazi militari base treasur hunter arctic photo amando flavious offici see link tru

# class of TextRank algorithm 

In [12]:
class TextRank():
    
    def __init__(self):
        self.d = 0.85 
        self.min_diff = 1e-5 
        self.node_weight = None 
    
    def sentence_segment(self, doc):
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    
    def get_token_pairs(self, window_size, sentences):
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
     # Get Symmeric matrix,we use a Symmeric matrix to represent the inbound and outbound links "we use the undirected edge"
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

 
    def get_keywords(self, number):
        x=[]
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        c=0
        for (key, value) in node_weight.items():
            if len(key)>1:
                x.append(key)
                print(key + ' - ' + str(value))
                c=c+1
            if c == (number-1):
                break
        return x
        
        
    def analyze(self, text, window_size):
        doc = nlp(text)
        sentences = self.sentence_segment(doc) 
        vocab = self.get_vocab(sentences)
        token_pairs = self.get_token_pairs(window_size, sentences)
        g = self.get_matrix(vocab, token_pairs)
        pr = np.array([1] * len(vocab))
        previous_pr = 0
        while True:
          pr = (1-self.d) + self.d * np.dot(g, pr)
          if abs(previous_pr - sum(pr))  < self.min_diff:
            break
          else:
            previous_pr = sum(pr)
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        self.node_weight = node_weight
        return node_weight
        

# Get all the words with their weights

In [13]:
tr = TextRank()
nodeWeight=tr.analyze(text, 4)
print(nodeWeight)

{'hous': 6.0291147360724295, 'dem': 1.3524781453928014, 'aid': 1.9821921695652756, 'even': 2.217765267396831, 'see': 2.938336821931018, 'comey': 4.107808849074437, 'letter': 1.5371425318463348, 'jason': 3.8141171032458123, 'chaffetz': 0.5818621988831849, 'tweet': 1.540714849745255, 'darrel': 0.7640482449235646, 'lucuflynn': 0.5298610266426573, 'hillari': 14.354312763307995, 'clinton': 14.26806680924405, 'big': 2.0595463100181224, 'woman': 3.143860851932711, 'campu': 0.5464197227827902, 'breitbart': 34.25389991574162, 'daniel': 4.902550575252652, 'j': 5.252543628381665, 'flynntruth': 0.5780670022626969, 'might': 1.0470801879650637, 'get': 7.049993572600878, 'fire': 4.473821296256215, 'consortiumnew': 1.1665987926825552, 'comcivilian': 0.6125162209695632, 'kill': 8.786866682937461, 'singl': 0.4628017789755422, 'us': 5.338958192938888, 'airstrik': 0.4922286322642175, 'identifi': 1.0529253203755196, 'jessica': 0.666994753333881, 'purkissiranian': 0.7438271880792812, 'jail': 1.4941194194819

# Getting the top keywords we need

In [14]:
top_keywords=tr.get_keywords(100)
print(top_keywords)
  

new - 59.8910458824623
york - 51.38644087941838
time - 46.37483492604783
trump - 43.393370955306395
breitbart - 34.25389991574162
hillari - 14.354312763307995
clinton - 14.26806680924405
elect - 10.94788069450001
say - 9.599107125591122
obama - 9.34558362270323
john - 9.207445789884684
kill - 8.786866682937461
de - 8.106097404405212
news - 7.879563748947715
day - 7.479723164729709
michael - 7.472483916442795
get - 7.049993572600878
state - 6.9676008342087234
war - 6.844301870138555
donald - 6.742270414781415
american - 6.693624342000859
world - 6.668297472235125
attack - 6.238270744993896
russia - 6.213989623542181
video - 6.138090993552026
plan - 6.090615353422708
vote - 6.090565757865826
hous - 6.0291147360724295
presid - 5.755108286365586
ian - 5.72819195708037
pam - 5.6412367223510484
illeg - 5.558961324900233
man - 5.444759913789166
life - 5.435939272574986
us - 5.338958192938888
go - 5.2556181039976915
first - 5.1980464072263155
david - 5.171877669613463
fight - 5.058750272286156

# Making the corpus

In [15]:

def MergeInList(DataSize):
    corpus=[]
    for i in range(DataSize):
        print(i)
        text= preProcessing(news_dataset['content'][i])
        corpus.append(text)
    print('fin')
    return corpus


In [16]:
corpus=MergeInList(1000)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [18]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet darrel lucu',
 'flynn hillari clinton big woman campu breitbart daniel j flynn',
 'truth might get fire consortiumnew com',
 'civilian kill singl us airstrik identifi jessica purkiss',
 'iranian woman jail fiction unpublish stori woman stone death adulteri howard portnoy',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart daniel nussbaum',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time alissa j rubin',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time megan twohey scott shane',
 'obama organ action partner soro link indivis disrupt trump agenda aaron klein',
 'bbc comedi sketch real housew isi caus outrag chri tomlinson',
 'russian research discov secret nazi militari base t

# Projection on df. We set the columns we want to display(top keywords)

In [17]:
#Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

dataFrame = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
X = dataFrame[top_keywords]

In [19]:
X

,new,york,time,trump,breitbart,hillari,clinton,elect,say,obama,john,kill,de,news,day,michael,get,state,war,donald,american,world,attack,russia,video,plan,vote,hous,presid,ian,pam,illeg,man,life,us,go,first,david,fight,report,...,take,right,found,muslim,one,govern,comey,paul,peopl,anti,daili,open,jason,arrest,secret,robert,care,todd,terror,face,border,protest,putin,home,group,work,eric,media,amanda,women,chri,ben,hollywood,voter,push,weiner,senat,top,china,migrant
0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.239527,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.255965,0.0,0.0,0.000000,0.0,0.000000,0.264569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.16994,0.210083,0.220414,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.351449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.264862,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.253603,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.000000,0.000000,0.000000,0.145347,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.271317,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.284904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.113169,0.116473,0.116303,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.272681,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.238125,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.257055,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [21]:
Y= news_dataset['label'].values
Y = Y[:1000]
Y

array([1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,

# Splitting the dataset to training and test data.
20% for testing ; 80% for training

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, stratify=Y, random_state=2)

# Training the model: Logistic Regression

In [23]:
model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# accurency score on the trainig and the testing data 

In [24]:

#score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy =accuracy_score(X_test_prediction, Y_test)
print("accurancy score of the testing data: ",test_data_accuracy)

accurancy score of the testing data:  0.915


# Compare the results of our model predictions with the actual result

In [25]:
# model predictions
for i in range(10):
    X_new = X_test[i:i+1]
    prediction = model.predict(X_new)
    if (prediction[0]==0):
        print(prediction,'The news is Real')
    else:
        print(prediction,'The news is Fake')

[0] The news is Real
[0] The news is Real
[1] The news is Fake
[0] The news is Real
[0] The news is Real
[0] The news is Real
[0] The news is Real
[1] The news is Fake
[1] The news is Fake
[1] The news is Fake


In [26]:
# Actual result
for i in range(10):
    print(Y_test[i])

0
0
0
0
0
0
0
1
1
1
